In [ ]:
import pandas as pd
import bibtexparser
import ast
import numpy as np

info = ['title', 'author', 'year', 'publisher', 'doi']

In [ ]:
with open(f"../Data/Mendeley Notes.bib") as bibtex_file:
    bib_database = bibtexparser.load(bibtex_file)

df = pd.DataFrame(bib_database.entries)

df = df.replace(r'\n',' ', regex=True) 
df['note'] = df['note'].str.replace('"',"'") 
df['note'] = df['note'].str.replace('\\',"") 
df['note'] = df['note'].str.replace('<br/>','')
df['note'] = df['note'].str.replace('obs','Obs')
df['note'] = df['note'].str.replace('comment','Obs')

In [ ]:
list_dict = []
i = 1

for index,row in df.iterrows():
    row_dict = ast.literal_eval(row['note'])

    for iinfo in info:
        row_dict[iinfo] = row[iinfo]

    if row_dict['id']!=i:
        i = row_dict['id']
    
    list_dict.append(row_dict)
    i += 1

notes_df = pd.DataFrame(list_dict)

In [ ]:
papers_df = pd.read_csv('../Data/Papers - After Abstract Filter.csv')

In [ ]:
papers_df['Title'] = papers_df['Title'].str.lower()
notes_df['title'] = notes_df['title'].str.lower()

In [ ]:
full_df = papers_df.join(notes_df.set_index('title'), on='Title')
full_df['id'].count()

In [ ]:
full_df[full_df['id'].isna()]

In [ ]:
full_df.to_csv('../Data/Papers - With Notes (incomplete).csv')

In [ ]:
manual_map = {'papers_id': [0, 14, 27, 139, 149, 159],
                'notes_id': [1, 12, 21, 121, 130, 139]}

In [ ]:
bib_ids_not_in_notesdf = [5, 19, 20, 22, 33, 53, 57, 69, 81, 88, 89, 94,
                        125, 128, 140, 155, 166]
papers_id_not_in_bib = [19, 95, 152]

In [ ]:
comp_df = pd.read_csv('../Data/Papers - With Notes (manually completed).csv')
#comp_df.drop('Unnamed: 0', axis=1, inplace=True)

comp_df = comp_df.replace(r'\n',' ', regex=True) 
col_notes = ['Optimization','ML task','Objective functions']
for col in col_notes:
    comp_df[col] = comp_df[col].str.replace('"',"'") 
    comp_df[col] = comp_df[col].str.replace('\\',"") 
    comp_df[col] = comp_df[col].str.replace('<br/>','')

comp_df.to_csv('../Data/Papers - With Notes (manually completed).csv', index=False)

In [ ]:
total = comp_df.shape[0]
mantidos = comp_df[comp_df['keep']==True].shape[0]
removidos = comp_df[comp_df['keep']==False].shape[0]
faltando = comp_df[comp_df['keep'].isnull()].shape[0]

print(f"Total de papers: {total}")
print(f"Papers mantidos: {mantidos}/{total} - {mantidos/total*100:.1f}%")
print(f"Papers removidos: {removidos}/{total} - {removidos/total*100:.1f}%")
print(f"Papers a analisar: {faltando}/{total} - {faltando/total*100:.1f}%")

In [ ]:
comp_df[comp_df['keep'].isnull()]

In [ ]:
sel_df = full_df[comp_df['keep']==True].copy()
sel_df.drop(['doi','keep','author','year','publisher','Obs','Remove - Abstract'], axis=1, inplace=True)
sel_df.to_csv('../Data/Papers - After Content Filter.csv', index=False)